In [11]:
import json, os, random

toolbench_data_path = '/cpfs/shared/research-llm/instruc_data_en/language_only_instruct_tuning/toolbench/data'

# samples = json.load(open(os.path.join(toolbench_data_path,'toolllama_G123_dfs_train.json')))
samples = json.load(open(os.path.join(toolbench_data_path,'toolllama_G123_dfs_train.json')))

print(len(samples))

26999


In [12]:
def print_sample(sample):
    # print(sample['id'])
    for content in sample['conversations']:
        system_message = "You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can't go back to one of the former state, if you want to restart the task, say \"I give up and restart\".\n2.All the thought is short, at most in 5 sentence.\n3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.\nLet's Begin!\nTask description: You should use functions to help handle the real time user querys. Remember:\n1.ALWAYS call \"Finish\" function at the end of the task. And the final answer should contain enough information to show to the user,If you can't handle the task, or you find that function calls always fail(the function is not valid now), use function Finish->give_up_and_restart.\n2.Do not use origin tool names, use only subfunctions' names.\n"
        print(f"🔥{content['from']}")
        print(content['value'].replace(system_message, ''))
        print('-'*64)


In [14]:
system_message = "You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can't go back to one of the former state, if you want to restart the task, say \"I give up and restart\".\n2.All the thought is short, at most in 5 sentence.\n3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.\nLet's Begin!\nTask description: You should use functions to help handle the real time user querys. Remember:\n1.ALWAYS call \"Finish\" function at the end of the task. And the final answer should contain enough information to show to the user,If you can't handle the task, or you find that function calls always fail(the function is not valid now), use function Finish->give_up_and_restart.\n2.Do not use origin tool names, use only subfunctions' names.\n"



def split_one_sample(sample):
    output_data = []
    user_tag = '### Human: '
    assistant_tag = '### Assistant: '

    conversation_history = ""
    for i, conversation in enumerate(sample['conversations']):
        if conversation['from'] in ['user', 'function', 'system']:
            if i==0:
                conversation_history = 'Toolbench System Message -- ' + conversation['value'].replace(system_message, '')
            else:
                conversation_history += f"\n{user_tag}{conversation['value']}"
            
        elif conversation['from'] == 'assistant':
            output = conversation['value']
            if output.startswith('\n'):
                output = output[1:]
            output_item = {
                'input': conversation_history.strip(),
                'output': output
            }
            output_data.append(output_item)
            conversation_history += f"\n{assistant_tag}{output}"
        
    return output_data

all_samples = []
os.makedirs('converted_datasets/toolbench', exist_ok=True)
print(len(samples))
for sample in samples:
    all_samples += split_one_sample(sample)
    
print(f'Writing to converted_datasets/toolbench/toolbench.json, total {len(all_samples)} items')
json.dump(all_samples, open('converted_datasets/toolbench/toolbench-train.json', 'w'), indent=4)

26999
Writing to converted_datasets/toolbench/toolbench.json, total 80210 items
